In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pl_bolts.datamodules import MNISTDataModule
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import wandb
wandb.login()

In [2]:
class Model(pl.LightningModule):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, self.config['hidden_size']),
            nn.ReLU(),
            nn.Linear(config['hidden_size'], 10),
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.config['learning_rate'])
        
    def compute_loss_and_accuracy(self, batch):

        x, y = batch
        logits = self(x)
        
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        accuracy = torchmetrics.functional.accuracy(preds, y)

        return loss, accuracy

    def training_step(self, batch, batch_idx):

        loss, accuracy = self.compute_loss_and_accuracy(batch)
        self.log("train_loss", loss)
        self.log("train_acc", accuracy)

        return loss

    def validation_step(self, batch, batch_idx):

        loss, accuracy = self.compute_loss_and_accuracy(batch)
        self.log("val_loss", loss)
        self.log("val_acc", accuracy)


In [3]:
sweep_config = {}
sweep_config['method'] = 'random'
sweep_config['metric'] = {'name': 'val_loss', 'goal': 'minimize'}
sweep_config['parameters'] = {'hidden_size': {'values': [128, 256, 512]},
                              'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},
                              'batch_size': {'values': [32, 64, 128]},}

sweep_id = wandb.sweep(sweep_config, project="test")

Create sweep with ID: kjcr1wl7
Sweep URL: https://wandb.ai/maximilienlc/test/sweeps/kjcr1wl7


In [4]:
def train_fn():

    with wandb.init():

        wandb_logger = WandbLogger(project="test")

        model = Model(wandb.config)

        data_module = MNISTDataModule(num_workers=8, batch_size=wandb.config['batch_size'])
        
        trainer = pl.Trainer(
            max_epochs=3,
            gpus=-1,
            logger=wandb_logger,
        )
        
        trainer.fit(model, data_module)

In [5]:
wandb.agent(sweep_id, train_fn, count=5)

wandb: Agent Starting Run: n167qfz8 with config:
wandb: 	batch_size: 128
wandb: 	hidden_size: 512
wandb: 	learning_rate: 0.03810167602541973
wandb: ERROR Failed to sample metric: Not Supported


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/datamodule.py:88: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/datamodule.py:107: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-----------------------------

Epoch 2: 100%|██████████| 469/469 [00:05<00:00, 84.41it/s, loss=0.236, v_num=qfz8]


wandb: ERROR Failed to serialize metric: division by zero


epoch,▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅████████
train_acc,▃▅▇▄▅▁▅▅▇▆▇▇█▇█▆▇▇▇▇▄█
train_loss,▆▃▃▆▂█▇▅▃▄▃▄▃▂▁▃▁▂▄▄▆▁
trainer/global_step,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇███
val_acc,▁█▃
val_loss,▁▆█
epoch,2
train_acc,0.96094
train_loss,0.11817
trainer/global_step,1124
val_acc,0.929


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 407 K 
-------------------------------------
407 K     Trainable params
0         Non-trainable params
407 K     Total params
1.628     Total estimated model params size (MB)


Epoch 1:  53%|█████▎    | 990/1875 [00:09<00:08, 99.59it/s, loss=0.228, v_num=w4cd] 

wandb: ERROR Failed to serialize metric: division by zero


Epoch 2: 100%|██████████| 1875/1875 [00:18<00:00, 100.66it/s, loss=0.175, v_num=w4cd] 


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅█████████████
train_acc,▁▆▅▁▅▆▆▅▅█▃▃▃▆▃▆▆▃▅█▃▅▆▅█▃███▆▃▆▆▃▆▃▅▁▅▆
train_loss,█▁▂▄▂▁▂▂▂▁▃▄▄▁▂▁▂▃▂▁▂▂▁▂▁▂▁▁▁▂▃▂▁▃▂▂▂▅▂▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▆█
val_loss,▁▂█
epoch,2
train_acc,0.96875
train_loss,0.07635
trainer/global_step,4499
val_acc,0.94875
